In [1]:
import pandas as pd
import json
import os
import websocket

cnt = 10
title = f"fall_detection_results_{cnt}"
subtitle = "ASVM: 4.5 (60) GSVM: 1.5 (30) angleY: 90 && angleX: 90 (20)"
# directories = ['./D01/', './D05/', './D12/', './D14/', './F01/', './F02/','./F08/','./F09/','./F13/','./F15/']
directories = ['./D01/']
ws_url = 'ws://localhost:8080/detection/fall?token={token}'
column_names = ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', 'acc2X', 'acc2Y', 'acc2Z']
acc_scaling_factor = 0.0383
gyro_scaling_factor = 0.01
results = []
results.append(subtitle)

def convert_txt_to_csv(txt_path, csv_path):
    try:
        df = pd.read_csv(txt_path, header=None, delimiter=',')
        df.columns = column_names
        df.to_csv(csv_path, index=False)
        print(f"Successfully converted {txt_path} to {csv_path}")
    except Exception as e:
        print(f"Failed to convert {txt_path} to {csv_path}: {e}")

def process_csv_file(csv_path):
    try:
        df = pd.read_csv(csv_path)
        
        if df.shape[1] < 3:
            raise ValueError(f"File {csv_path} does not contain enough columns.")

        # Scaling accelerometer and gyroscope data
        df[['accX', 'accY', 'accZ']] = df[['accX', 'accY', 'accZ']] * acc_scaling_factor
        df[['gyroX', 'gyroY', 'gyroZ']] = df[['gyroX', 'gyroY', 'gyroZ']] * gyro_scaling_factor

        # Rounding the values for better readability
        df = df.round({'accX': 2, 'accY': 2, 'accZ': 2, 'gyroX': 2, 'gyroY': 2, 'gyroZ': 2})
        
        fall_data = df.iloc[:, :6]
        fall_data.columns = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ"]
        fall_data = fall_data.to_dict(orient="records")
        
        return fall_data

    except Exception as e:
        print(f"Failed to process {csv_path}: {e}")
        return None

def split_data(data, chunk_size=80):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

def send_json_via_websocket(json_data, url):
    try:
        ws = websocket.WebSocket()
        ws.connect(url)
        ws.send(json.dumps(json_data))
        response = ws.recv()
        ws.close()
        
        response_data = json.loads(response)
        if response_data.get("event") == "fall":
            success = response_data.get("success")
            return success
        else:
            print(f"Unexpected response: {response}")
            return False
            
    except Exception as e:
        print(f"Failed to send data via WebSocket: {e}")
        return False

for directory in directories:
    print(f"{directory}-------------------------------------------------------------------------------")
    txt_file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.txt')]
    total_cnt = 0
    total_success = 0
    total_zero = 0
    total_group = 0
    for txt_file_path in txt_file_paths:
        count = 0
        success_cnt = 0
        csv_file_path = txt_file_path.replace('.txt', '.csv')
        convert_txt_to_csv(txt_file_path, csv_file_path)
        
        fall_data = process_csv_file(csv_file_path)
        
        if fall_data is None:
            continue
        
        chunks = list(split_data(fall_data, chunk_size=80))
        
        for idx, chunk in enumerate(chunks):
            json_data = {
                "event": "fall",
                "data": {
                    "fall": chunk
                }
            }
            
            json_file_path = txt_file_path.replace('.txt', f'_{idx}.json')
            with open(json_file_path, 'w') as f:
                json.dump(json_data, f, indent=4)
            
            success = send_json_via_websocket(json_data, ws_url)
            count += 1
            if success:
                success_cnt += 1
        print(f"{success_cnt}/{count}")
        total_group += 1
        if success_cnt==0:
            total_zero += 1
        total_success += success_cnt
        total_cnt += count
    result = f"{directory} fall_detected_count={total_success}/{total_cnt}, group_zero_count={total_zero}/{total_group}"
    print(result)
    results.append(result)

with open(title, 'w') as f:
    for result in results:
        f.write(result + '\n')

print("End")


./D01/-------------------------------------------------------------------------------
Successfully converted ./D01/D01_SA14_R01.txt to ./D01/D01_SA14_R01.csv
0/250
Successfully converted ./D01/D01_SE02_R01.txt to ./D01/D01_SE02_R01.csv
0/250
Successfully converted ./D01/D01_SA15_R01.txt to ./D01/D01_SA15_R01.csv
0/250
Successfully converted ./D01/D01_SA17_R01.txt to ./D01/D01_SA17_R01.csv
0/250
Successfully converted ./D01/D01_SE01_R01.txt to ./D01/D01_SE01_R01.csv
0/450
Successfully converted ./D01/D01_SA16_R01.txt to ./D01/D01_SA16_R01.csv
0/250
Successfully converted ./D01/D01_SA13_R01.txt to ./D01/D01_SA13_R01.csv
0/250
Successfully converted ./D01/D01_SA12_R01.txt to ./D01/D01_SA12_R01.csv
0/250
Successfully converted ./D01/D01_SA09_R01.txt to ./D01/D01_SA09_R01.csv
0/250
Successfully converted ./D01/D01_SA10_R01.txt to ./D01/D01_SA10_R01.csv
0/250
Successfully converted ./D01/D01_SA11_R01.txt to ./D01/D01_SA11_R01.csv
0/250
Successfully converted ./D01/D01_SA18_R01.txt to ./D01/D

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# 파일 경로
file_path = "./fall_detection_results_9"

# 데이터 읽기 및 파싱
directories = []
fall_detected_counts = []
total_counts = []
group_zero_counts = []
total_groups = []

with open(file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if "fall_detected_count" in line:
            parts = line.split()
            directory = parts[0]
            fall_detected_count = int(parts[1].split('=')[1].split('/')[0])
            total_count = int(parts[1].split('=')[1].split('/')[1].split(',')[0])
            group_zero_count = int(parts[2].split('=')[1].split('/')[0])
            total_group = int(parts[2].split('=')[1].split('/')[1])
            
            directories.append(directory)
            fall_detected_counts.append(fall_detected_count)
            total_counts.append(total_count)
            group_zero_counts.append(group_zero_count)
            total_groups.append(total_group)

# 데이터프레임 생성
data = {
    "directory": directories,
    "fall_detected_count": fall_detected_counts,
    "total_count": total_counts,
    "group_zero_count": group_zero_counts,
    "total_group": total_groups
}

df = pd.DataFrame(data)

# 시각화 및 저장

# 낙상 감지 횟수 시각화
# plt.figure(figsize=(12, 6))
# plt.bar(df['directory'], df['fall_detected_count'], color='blue', alpha=0.7, label='Fall Detected Count')
# plt.xlabel('Directory')
# plt.ylabel('Fall Detected Count')
# plt.title('Fall Detected Count per Directory')
# plt.legend()
# plt.savefig(f'./fall_detected_count_per_directory_{cnt}.png')
# plt.show()

# 그룹 제로 카운트 시각화
df['group_zero_ratio'] = df['group_zero_count'] / df['total_group']
print(df)
plt.figure(figsize=(12, 6))
plt.bar(df['directory'], df['group_zero_ratio'], color='red', alpha=0.7, label='Group Zero Count')
plt.xlabel('Directory')
plt.ylabel('Group Zero Ratio')
plt.title('Group Zero Count ratio Directory')
plt.legend()
plt.savefig(f'./group_zero_ratio_per_directory_{cnt}.png')
plt.show()

# 낙상 감지 비율 시각화
df['fall_detected_ratio'] = df['fall_detected_count'] / df['total_count']
plt.figure(figsize=(12, 6))
plt.bar(df['directory'], df['fall_detected_ratio'], color='green', alpha=0.7, label='Fall Detected Ratio')
plt.xlabel('Directory')
plt.ylabel('Fall Detected Ratio')
plt.title('Fall Detected Ratio per Directory')
plt.legend()
plt.savefig(f'./fall_detected_ratio_per_directory_{cnt}.png')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: './fall_detection_results_9'